<a href="https://colab.research.google.com/github/rainmaker29/Tensorflow-Lite/blob/master/TFLITE_Cats%26Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("TF version : ",tf.__version__)
print("Eager Execution : ",tf.executing_eagerly())
print("Hub version : ",hub.__version__)
print("GPU is"," available " if tf.test.is_gpu_available() else " unavailable")

TF version :  2.2.0-rc2
Eager Execution :  True
Hub version :  0.8.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is  available 


In [0]:
module_selection = ("inception_v3", 299, 2048) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(
  MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4 with input size (299, 299) and output dimension 2048


In [0]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [0]:
(train_examples ,validation_examples,test_examples) , info = tfds.load(
    'cats_vs_dogs',
    split=['train[80%:]','train[80%:90%]','train[90%:]'],
    with_info = True,
    as_supervised=True,)


num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [0]:
#Resizing images
def format_image(image,label):
  image = tf.image.resize(image,IMAGE_SIZE)/255.0
  return image,label

In [0]:
BATCH_SIZE = 32


In [0]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [0]:
for image_batch,label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 299, 299, 3])

In [0]:
do_fine_tuning = False #@param {type:"boolean"}

In [0]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape = IMAGE_SIZE+(3,),
                                   output_shape=[FV_SIZE],
                                   trainable=do_fine_tuning)

In [0]:
print("Model used : ",MODULE_HANDLE)

model = tf.keras.models.Sequential([
          
          feature_extractor,
          tf.keras.layers.Dense(num_classes)
])

model.summary()

Model used :  https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


In [0]:
#@title (Optional) Unfreeze some layers
NUM_LAYERS = 7 #@param {type:"slider", min:1, max:50, step:1}
      
if do_fine_tuning:
  feature_extractor.trainable = True
  
  for layer in model.layers[-NUM_LAYERS:]:
    layer.trainable = True

else:
  feature_extractor.trainable = False

In [0]:
if do_fine_tuning:
  model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.002, momentum=0.9), 
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
else:
  model.compile(
    optimizer='adam', 
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [0]:
EPOCHS = 5
hist = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/5
146/146 [==============================] - 35s 241ms/step - loss: 0.0982 - accuracy: 0.9729 - val_loss: 0.0329 - val_accuracy: 0.9923
Epoch 2/5
146/146 [==============================] - 32s 222ms/step - loss: 0.0262 - accuracy: 0.9944 - val_loss: 0.0206 - val_accuracy: 0.9940
Epoch 3/5
146/146 [==============================] - 33s 223ms/step - loss: 0.0185 - accuracy: 0.9942 - val_loss: 0.0148 - val_accuracy: 0.9974
Epoch 4/5
146/146 [==============================] - 33s 226ms/step - loss: 0.0141 - accuracy: 0.9959 - val_loss: 0.0117 - val_accuracy: 0.9983
Epoch 5/5
146/146 [==============================] - 33s 227ms/step - loss: 0.0112 - accuracy: 0.9970 - val_loss: 0.0095 - val_accuracy: 0.9983


In [0]:
CATS_VS_DOGS_SAVED_MODEL = "exp_saved_model"
tf.saved_model.save(model,CATS_VS_DOGS_SAVED_MODEL)
loaded = tf.saved_model.load(CATS_VS_DOGS_SAVED_MODEL)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: exp_saved_model/assets


INFO:tensorflow:Assets written to: exp_saved_model/assets


In [0]:
#USING TFLITE CONVERTER

In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)

In [0]:
#Quantization
#Using default mode so that every value converts to int for faster calculations

# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

In [0]:
#Representative datasets to transform the weights
# def representative_data_gen():
#   for input_value, _ in test_batches.take(100):
#     yield [input_value]

In [0]:
# converter.representative_dataset = representative_data_gen

In [0]:
# The above implementation will compute calculations in integers while those which cannot be
# made int are carried out as floats. Also the input and output will be floating values for convinience

In [0]:
#To get strict int operations and outputs
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

In [0]:
#Finally converting the model
tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)

In [0]:
# Load TFLite model and allocate tensors.
  
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [63]:
from tqdm import tqdm

# Gather results for the randomly sampled test images
predictions = []

test_labels, test_imgs = [], []
for img, label in tqdm(test_batches.take(10)):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  
  test_labels.append(label.numpy()[0])
  test_imgs.append(img)

10it [00:02,  4.29it/s]


In [0]:
!mkdir -p test_images

In [0]:
class_names=['cat','dog']
from PIL import Image

for index, (image, label) in enumerate(test_batches.take(50)):
  image = tf.cast(image * 255.0, tf.uint8)
  image = tf.squeeze(image).numpy()
  pil_image = Image.fromarray(image)
  pil_image.save('test_images/{}_{}.jpg'.format(class_names[label[0]], index))

In [67]:
!ls test_images

cat_0.jpg   cat_21.jpg	cat_32.jpg  cat_45.jpg	dog_13.jpg  dog_41.jpg
cat_11.jpg  cat_22.jpg	cat_34.jpg  cat_46.jpg	dog_17.jpg  dog_44.jpg
cat_12.jpg  cat_23.jpg	cat_35.jpg  cat_47.jpg	dog_1.jpg   dog_48.jpg
cat_14.jpg  cat_24.jpg	cat_36.jpg  cat_4.jpg	dog_27.jpg  dog_49.jpg
cat_15.jpg  cat_25.jpg	cat_39.jpg  cat_5.jpg	dog_29.jpg  dog_6.jpg
cat_16.jpg  cat_26.jpg	cat_3.jpg   cat_7.jpg	dog_2.jpg
cat_18.jpg  cat_28.jpg	cat_40.jpg  cat_8.jpg	dog_33.jpg
cat_19.jpg  cat_30.jpg	cat_42.jpg  cat_9.jpg	dog_37.jpg
cat_20.jpg  cat_31.jpg	cat_43.jpg  dog_10.jpg	dog_38.jpg


In [0]:
!zip -qq cats_vs_dogs_test_images.zip -r test_images/

In [0]:
try:
  files.download('cats_vs_dogs_test_images.zip')
except:
  pass